In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1TOHrzIpiJQkHLJEPW62l-KtEmOvX698jh7nHfLSUzdg/export?format=csv"
df = pd.read_csv(url)
df.head()

,項目別,國中,高級中等,專科,大學,年增率,研究所,年增率_1
0,NaN,NaN,（高中、高職）,NaN,NaN,(%),NaN,(%)
1,全體,28,29,32,34.0,5.1,52,4.3
2,性別,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,男性,28,30,32,34.0,5,54,3.8
4,女性,29,29,32,34.0,5.1,48,5.2


In [ ]:
# 移除列
df = df.drop(index=[0, 2, 5, 6, 9])
# 重設索引
df = df.reset_index(drop=True)
# 刪除年增率相關欄
df = df.drop(columns=["年增率", "年增率_1"], errors="ignore")
# 將 '---' 或 '-' 換成 NaN
df = df.replace(['---', '--', '-', '—'], np.nan)
# 將數值欄位轉為 float
for col in ["國中", "高級中等", "專科", "大學", "研究所"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [ ]:
df

,項目別,國中,高級中等,專科,大學,研究所
0,全體,28.0,29.0,32.0,34.0,52.0
1,男性,28.0,30.0,32.0,34.0,54.0
2,女性,29.0,29.0,32.0,34.0,48.0
3,中小企業,28.0,29.0,30.0,31.0,42.0
4,大企業及其他,30.0,31.0,35.0,38.0,55.0
5,農、林、漁、牧業,NaN,28.0,NaN,31.0,33.0
6,礦業及土石採取業,NaN,NaN,NaN,33.0,NaN
7,製造業,28.0,29.0,31.0,34.0,58.0
8,電力及燃氣供應業,NaN,31.0,NaN,43.0,43.0
9,用水供應及污染整治業,NaN,30.0,NaN,32.0,38.0


In [ ]:
# 欄位名稱中 → 英文對應
df = df.rename(columns={
    "項目別": "Category",
    "國中": "Junior High",
    "高級中等": "Senior High",
    "專科": "Junior College",
    "大學": "University",
    "研究所": "Graduate School"
})

In [ ]:
# 中文產業名稱對照表
industry_map = {
    "全體": "All Employees",
    "男性": "Male",
    "女性": "Female",
    "中小企業": "Small & Medium Enterprises",
    "大企業及其他": "Large Enterprises & Others",
    "農、林、漁、牧業": "Agriculture, Forestry, Fishing & Animal Husbandry",
    "礦業及土石採取業": "Mining & Quarrying",
    "製造業": "Manufacturing",
    "電力及燃氣供應業": "Electricity & Gas Supply",
    "用水供應及污染整治業": "Water Supply & Waste Management",
    "營建工程業": "Construction",
    "批發及零售業": "Wholesale & Retail Trade",
    "運輸及倉儲業": "Transportation & Storage",
    "住宿及餐飲業": "Accommodation & Food Service",
    "出版影音及資通訊業": "Publishing, Audio-Visual & Information Communication",
    "金融及保險業": "Finance & Insurance",
    "不動產業": "Real Estate",
    "專業、科學及技術服務業": "Professional, Scientific & Technical Services",
    "支援服務業": "Administrative & Support Services",
    "公共行政及國防；強制性社會安全": "Public Administration & Defense; Compulsory Social Security",
    "教育業": "Education",
    "醫療保健及社會工作服務業": "Health Care & Social Work",
    "藝術、娛樂及休閒服務業": "Arts, Entertainment & Recreation",
    "其他服務業": "Other Services"
}

# 將 Category 欄位內中文轉成英文
df["Category"] = df["Category"].replace(industry_map)


In [ ]:
def get_category_type(category):
    if category == "All Employees":
        return "Total"
    elif category in ["Male", "Female"]:
        return "Gender"
    elif category in ["Small & Medium Enterprises", "Large Enterprises & Others"]:
        return "Enterprise Size"
    else:
        return "Industry"

df['Category_Type'] = df['Category'].apply(get_category_type)

# Reorder columns to place 'Category_Type' after 'Category'
cols = df.columns.tolist()
cols.insert(cols.index('Category') + 1, cols.pop(cols.index('Category_Type')))
df = df[cols]

In [ ]:
# 匯率設定（用 1 USD = 32.0 TWD，依台灣近年平均匯率）
twd_to_usd = 1 / 32.0
for col in ["Junior High", "Senior High", "Junior College", "University", "Graduate School"]:
    df[col] = pd.to_numeric(df[col], errors="coerce") * 1000 * twd_to_usd


In [ ]:
# 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 定義儲存檔案的路徑和名稱
# 使用您提供的實際資料夾路徑
output_path = '/content/drive/My Drive/Colab Notebooks/wage/df_tw_wide.csv'

# 將 wide DataFrame 輸出為 CSV 檔案
# index=False 表示不將 DataFrame 的索引寫入 CSV 檔案中
df.to_csv(output_path, index=False)

print(f"wide DataFrame 已成功儲存到 {output_path}")

Mounted at /content/drive
wide DataFrame 已成功儲存到 /content/drive/My Drive/Colab Notebooks/wage/df_tw_wide.csv


In [ ]:
df

,Category,Category_Type,Junior High,Senior High,Junior College,University,Graduate School
0,All Employees,Total,875.00,906.25,1000.00,1062.50,1625.00
1,Male,Gender,875.00,937.50,1000.00,1062.50,1687.50
2,Female,Gender,906.25,906.25,1000.00,1062.50,1500.00
3,Small & Medium Enterprises,Enterprise Size,875.00,906.25,937.50,968.75,1312.50
4,Large Enterprises & Others,Enterprise Size,937.50,968.75,1093.75,1187.50,1718.75
5,"Agriculture, Forestry, Fishing & Animal Husbandry",Industry,NaN,875.00,NaN,968.75,1031.25
6,Mining & Quarrying,Industry,NaN,NaN,NaN,1031.25,NaN
7,Manufacturing,Industry,875.00,906.25,968.75,1062.50,1812.50
8,Electricity & Gas Supply,Industry,NaN,968.75,NaN,1343.75,1343.75
9,Water Supply & Waste Management,Industry,NaN,937.50,NaN,1000.00,1187.50


In [ ]:
# 將 wide DataFrame 轉為 long DataFrame
df_long = df.melt(
    id_vars=["Category", "Category_Type"],
    var_name="Education",
    value_name="Salary"
)

# 顯示 long DataFrame
display(df_long.head())

,Category,Category_Type,Education,Salary
0,All Employees,Total,Junior High,875.00
1,Male,Gender,Junior High,875.00
2,Female,Gender,Junior High,906.25
3,Small & Medium Enterprises,Enterprise Size,Junior High,875.00
4,Large Enterprises & Others,Enterprise Size,Junior High,937.50


In [ ]:
ppp_conversion_factor = 13.66
twd_exchange_rate = 1 / twd_to_usd # This is 32.0
df_long['Real Salary'] = (df_long['Salary'] * twd_exchange_rate) / ppp_conversion_factor

display(df_long.head())

,Category,Category_Type,Education,Salary,Real Salary
0,All Employees,Total,Junior High,875.00,2049.780381
1,Male,Gender,Junior High,875.00,2049.780381
2,Female,Gender,Junior High,906.25,2122.986823
3,Small & Medium Enterprises,Enterprise Size,Junior High,875.00,2049.780381
4,Large Enterprises & Others,Enterprise Size,Junior High,937.50,2196.193265


In [ ]:
numeric_cols = ["Junior High", "Senior High", "Junior College", "University", "Graduate School"]
df_real = df.copy()
df_real[numeric_cols] = df_real[numeric_cols] * twd_exchange_rate / ppp_conversion_factor
df_real

,Category,Category_Type,Junior High,Senior High,Junior College,University,Graduate School
0,All Employees,Total,2049.780381,2122.986823,2342.606149,2489.019034,3806.734993
1,Male,Gender,2049.780381,2196.193265,2342.606149,2489.019034,3953.147877
2,Female,Gender,2122.986823,2122.986823,2342.606149,2489.019034,3513.909224
3,Small & Medium Enterprises,Enterprise Size,2049.780381,2122.986823,2196.193265,2269.399707,3074.670571
4,Large Enterprises & Others,Enterprise Size,2196.193265,2269.399707,2562.225476,2781.844802,4026.354319
5,"Agriculture, Forestry, Fishing & Animal Husbandry",Industry,NaN,2049.780381,NaN,2269.399707,2415.812592
6,Mining & Quarrying,Industry,NaN,NaN,NaN,2415.812592,NaN
7,Manufacturing,Industry,2049.780381,2122.986823,2269.399707,2489.019034,4245.973646
8,Electricity & Gas Supply,Industry,NaN,2269.399707,NaN,3147.877013,3147.877013
9,Water Supply & Waste Management,Industry,NaN,2196.193265,NaN,2342.606149,2781.844802


In [ ]:
output_path = '/content/drive/My Drive/Colab Notebooks/wage/df_tw_long.csv'
df_long.to_csv(output_path, index=False)